In [6]:
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.engine import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.schema import Column, ForeignKey
from sqlalchemy.types import Integer, Text
from sqlalchemy import Table

In [19]:
engine = create_engine('sqlite:///:memory:', echo=True)
session = sessionmaker(engine)
sess = session()
base = declarative_base()

In [4]:
from sqlalchemy import MetaData

meta_obj = MetaData()

In [20]:
class Artist(base):
    __tablename__ = 'T_ARTIST'
    __table_args__ = {'extend_existing': True}

    pk = Column("PK", Integer, primary_key = True)
    name = Column("NAME", Text)
    albums = relationship('Album', back_populates='artists', uselist=True)

    def addAlbum(self, s, name):
        a = s.query(Artist).filter(Artist.pk==self.pk).one()
        a.albums.append(Album(name=name))
        s.commit()
    
    def delAlbum(self, s, name):
        a = s.query(Artist).filter(Artist.pk==self.pk).one()
        a.albums.remove(list(filter(self.name==name, self.albums))[0])
        s.commit()

    def __repr__(self):
        return f'pk={self.pk}, name={self.name}'
    
class Album(base):
    __tablename__ = 'T_ALBUM'
    __table_args__ = {'extend_existing': True}
    
    pk = Column("PK", Integer, primary_key = True)
    name = Column("NAME", Text)
    fk = Column("FK", None, ForeignKey(Artist.pk))
    artists = relationship('Artist', back_populates='albums', uselist=False)

    def __repr__(self):
        return f'pk={self.pk}, name={self.name}'

In [21]:
base.metadata.create_all(engine)

2024-03-18 19:45:18,106 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-18 19:45:18,107 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_ARTIST")
2024-03-18 19:45:18,107 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-18 19:45:18,109 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_ARTIST")
2024-03-18 19:45:18,109 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-18 19:45:18,110 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("T_ALBUM")
2024-03-18 19:45:18,111 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-18 19:45:18,112 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("T_ALBUM")
2024-03-18 19:45:18,112 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-18 19:45:18,114 INFO sqlalchemy.engine.Engine 
CREATE TABLE "T_ARTIST" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT, 
	PRIMARY KEY ("PK")
)


2024-03-18 19:45:18,115 INFO sqlalchemy.engine.Engine [no key 0.00070s] ()
2024-03-18 19:45:18,116 INFO sqlalchemy.engine.Engine 
CREATE TABLE "T_ALBUM" (
	"PK" I

In [22]:
a = Artist(name='가수1')

In [23]:
sess.add(a)
sess.commit()

2024-03-18 19:46:46,076 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-18 19:46:46,079 INFO sqlalchemy.engine.Engine INSERT INTO "T_ARTIST" ("NAME") VALUES (?)
2024-03-18 19:46:46,080 INFO sqlalchemy.engine.Engine [generated in 0.00094s] ('가수1',)
2024-03-18 19:46:46,081 INFO sqlalchemy.engine.Engine COMMIT


In [24]:
a.addAlbum(sess, '가수1의 앨범1')

2024-03-18 19:47:27,583 INFO sqlalchemy.engine.Engine SELECT "T_ARTIST"."PK" AS "T_ARTIST_PK", "T_ARTIST"."NAME" AS "T_ARTIST_NAME" 
FROM "T_ARTIST" 
WHERE "T_ARTIST"."PK" = ?
2024-03-18 19:47:27,584 INFO sqlalchemy.engine.Engine [generated in 0.00082s] (1,)
2024-03-18 19:47:27,587 INFO sqlalchemy.engine.Engine INSERT INTO "T_ALBUM" ("NAME", "FK") VALUES (?, ?)
2024-03-18 19:47:27,588 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ('가수1의 앨범1', 1)
2024-03-18 19:47:27,589 INFO sqlalchemy.engine.Engine COMMIT


In [26]:
sess.query(Album).all()

2024-03-18 19:49:22,251 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-18 19:49:22,253 INFO sqlalchemy.engine.Engine SELECT "T_ALBUM"."PK" AS "T_ALBUM_PK", "T_ALBUM"."NAME" AS "T_ALBUM_NAME", "T_ALBUM"."FK" AS "T_ALBUM_FK" 
FROM "T_ALBUM"
2024-03-18 19:49:22,254 INFO sqlalchemy.engine.Engine [generated in 0.00130s] ()


[pk=1, name=가수1의 앨범1]

In [18]:
sess.close()
base.registry.dispose()
base.metadata.clear()
engine.dispose()